# 1) Données

In [30]:
import pandas as pd
import numpy as np
import csv
import os

# calcul de la position dans le roulement d'un poste en fonction de la semaine
def rFunction(s,p):
    return ((p-s)%36)

# calcul du numéro de semaine correspondant au jour j
def sFunction(j):
    return j//7

# calcul du jour non travaillé pour une semaine donnée d'un opérateur à 80%
# en fonction du jour non travaillé la première semaine
def delta_i(s,d_i):
    return (d_i+s)%5


cardO = 36
cardP = 36

rho = [1,1,1,1,0,1,0,1,1,1,1,1,1,0,1,1,1,0,1,0,1,1,1,0,1,1,1,0,0,1,1,0,1,1,0,1]

# le nombre de semaines <=> nombre de postes <=> nombre d'opérateurs
#cardS = cardO
cardS = 1
cardJ = 7*cardS
cardR = cardO

# kappa_ip (opérateur possède la compétence pour le poste)
#kappa = pd.read_csv('./data/kappa_ip.csv',sep = ";" )
kappa = [
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[0,0,0,1,0,1,1,0,0,0,1,0,0,1,0,1,0,1,1,0,0,0,1,1,0,1,1,1,1,0,0,0,0,1,1,0],
[0,0,0,1,0,1,1,0,0,0,1,0,0,1,0,1,0,1,1,0,0,0,1,1,0,1,1,1,1,0,0,0,0,1,1,0],
[1,0,0,1,0,1,1,0,0,1,1,0,1,1,0,1,0,1,1,0,0,0,1,1,0,1,1,1,1,0,0,0,0,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[1,0,0,1,0,1,1,0,0,1,1,0,1,1,1,1,0,1,1,0,0,0,1,1,0,1,1,1,1,0,0,0,0,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[0,0,0,1,0,1,1,0,0,0,1,0,0,1,0,1,0,1,1,0,0,0,1,1,0,1,1,1,1,0,0,0,0,1,1,0],
[1,0,0,1,0,1,1,0,0,1,1,0,1,1,1,1,0,1,1,0,0,0,1,1,0,1,1,1,1,0,0,0,0,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[1,1,0,1,0,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0],
[0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1,1,0,1,0,1,1,0,0,0,0,0,1,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[0,1,1,0,1,1,1,1,1,0,1,1,0,1,0,1,1,1,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[0,0,1,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[0,0,1,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1],
[0,0,1,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[0,0,1,0,1,1,0,1,1,0,0,1,0,0,0,1,0,0,0,1,0,1,1,0,1,1,0,0,0,1,0,1,0,0,0,1],
[0,0,1,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1],
[0,0,1,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1]]

# sigma_pp' (postes sur le même créneau horaire)
#Créneaux horaires (0=rouleur, 1=7h, 2=10h30, 3=12h45)
creneaux = [5,1,4,1,0,4,0,4,3,4,5,4,1,0,4,1,4,0,1,0,5,4,1,0,3,4,1,0,1,4,5,0,1,4,0,2]
sigma = []
for p in range(len(creneaux)):
    ligne = []
    creneauP = creneaux[p]
    for p2 in range(len(creneaux)):
        creneauP2 = creneaux[p2]
        if creneauP == creneauP2 or creneauP == 0 or creneauP2 == 0:
            ligne.append(1)
        else:
            ligne.append(0)
    sigma.append(ligne)

# delta_ij (est-ce qu'une personne à 80% travaille le jour j)
#delta = pd.read_csv('./data/delta_ij.csv',sep = ";" )
# d[i] représente le jour non travaillé de l'opérateur i la première semaine, d[i]=-1 si i est à temps plein
d = []
for i in range(36):
    d.append(-1)

#postes facultatifs -> cond1-7h, cond2-7h et REC-10h30-Uro/Visc/cpp
facIndex = [22,15,2]
fac = []
for i in range(cardP):
    if i in facIndex:
        fac.append(1)
    else:
        fac.append(0)

delta = np.full((cardO, cardJ), 1)
for i in range(0,cardO):
    if d[i] != -1:
        d_i = d[i]
        for s in range(0, cardS):
            d_is = delta_i(s,d_i) #rang du jour non travaillé la semaine s
            j_s = 7*s + d_is      #numéro du jour non travaillé
            delta[i][j_s] = 0

# o_j (jour ouvrés)
o = np.zeros(cardJ)
for j in range(0,len(o)):
    if (j%7 <= 4):
        o[j] = 1

# H_s (jours de chaque semaine)
H = []
for i in range(0,cardS):
    H.append(np.zeros(7))
for i in range(0,cardS):
    for j in range(0,7):
        H[i][j]=(i*7 + j)
        
profils = np.zeros(cardO)
cardProfils = 0
for i in range(0,cardO):
    found = False
    for j in range(0,i):
        if not found:
            diff = False
            if d[i] == d[j]:
                for p in range(0,cardP):
                    if kappa[i][p] != kappa[j][p]:
                        diff = True
            else:
                diff = True
        
            if not diff:
                profil = profils[j]
                found = True
    if found:
        profils[i] = profil
    else:
        cardProfils = cardProfils + 1
        profils[i] = cardProfils
        
poids1 = 1
poids2 = 10
poids3 = 100
poids4 = 1000

#print(nomsOperateurs)
#print(competencesOperateurs)
#print(nomsPostes)
#print(rho)
#print(kappa)
for i in range(cardP):
    print(sigma[i])
#print(delta)
#print(o)
#print(H[1])
print("fin")

[1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0]
[0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0]
[0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0]
[0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0]
[0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0]
[0, 0, 1, 0, 1, 1, 

# 2) Modèle mathématique


## 2.1) Variables

In [31]:
from pulp import *

# x_ijp = 1 si l'opérateur prend en charge le poste p le jour j
x = {} 
for i in range(0,cardO):
    for j in range(0,cardJ): 
        for p in range(0,cardP): 
            x[i,j,p]=LpVariable(f"x({i,j,p})", cat=LpBinary) #'Continuous'

# y_ir = 1 si l'opérateur i est position r dans le roulement
y = {} 
for i in range(0,cardO):
    for r in range(0,cardR):    
        y[i,r]=LpVariable(f"y({i,r})", cat=LpBinary)
        
# z_isp = 1 si l'opérateur est affecté au poste p la semaine s
z = {} 
for i in range(0,cardO):
    for s in range(0,cardS): 
        for p in range(0,cardP): 
            z[i,s,p]=LpVariable(f"z({i,s,p})", cat=LpBinary)

# u_is = 1 si l'opérateur i est affecté à un poste différent de celui prévu la semaine s
u = {} 
for i in range(0,cardO):
    for s in range(0,cardS):    
        u[i,s]=LpVariable(f"u({i,s})", cat=LpBinary)
        
u1 = {} 
for i in range(0,cardO):
    for s in range(0,cardS):    
        u1[i,s]=LpVariable(f"u1({i,s})", cat=LpBinary)
        
u2 = {} 
for i in range(0,cardO):
    for s in range(0,cardS):    
        u2[i,s]=LpVariable(f"u2({i,s})", cat=LpBinary)
        
posteVacant = {} 
for p in range(0,cardP):
    for s in range(0,cardS):    
        posteVacant[p,s]=LpVariable(f"posteVacant({p,s})", cat=LpBinary)

print("fin")

fin


## 2.2) Création du modèle avec PuLP

In [32]:
prob = LpProblem("ModeleExact", LpMinimize)

prob += poids1*sum(u[i,s] for i in range(0,cardO) for s in range(0,cardS)) + poids2*sum(u1[i,s] for i in range(0,cardO) for s in range(0,cardS)) + poids3*sum(posteVacant[p,s] for p in range(0,cardP) for s in range(0,cardS)) + poids4*sum(u2[i,s] for i in range(0,cardO) for s in range(0,cardS)), "obj"

## 2.3) Contraintes

In [33]:
tempY = [[0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]]

for i in range(cardO):
    for r in range(cardR):
        a = 0
        #prob += y[i,r] == tempY[i][r]

#contrainte 1 Un opérateur ne peut prendre un poste que s'il est compétent et disponible sur un jour ouvré

for i in range(0,cardO):
    for j in range(0,cardJ):
        for p in range(0,cardP):
            prob += (x[i,j,p] <= delta[i][j]*kappa[i][p]*o[j], f"C1_{i,j,p}")

#contrainte 2 Un opérateur ne peut être affecté à un poste une semaine que s'il en possède la compétence

for i in range(0,cardO):
    for s in range(0,cardS):
        for p in range(0,cardP):
            prob += (z[i,s,p] <= kappa[i][p], f"C2_{i,s,p}")

#contrainte 3 Chaque jour, chaque poste est effectué par l'opé y étant affecté cette semaine ou par un rouleur
for i in range(0,cardO):
    for s in range(0,cardS):
        for j in H[s]:
            for p in range(0,cardP):
                prob += x[i,j,p] <= z[i,s,p] + sum((1 - rho[p2])*z[i,s,p2] for p2 in range(0,cardP)), f"C3_{i,j,p}"

#contrainte 4 Chaque jour, un opérateur est affecté à au plus un poste

for i in range(0,cardO):
    for j in range(0,cardJ):
        if o[j] == 1 and delta[i][j] == 1:
            prob += sum(x[i,j,p] for p in range(0,cardP)) == 1, f"C4_{i,j}"

#contrainte 5 Chaque jour, chaque poste est assuré par un opérateur
for p in range(0,cardP):
    for j in range(0,cardJ):
        if o[j] == 1 and rho[p] == 1 and fac[p] == 0:
            prob += sum(x[i,j,p] for i in range(0,cardO)) == 1, f"C5_{p,j}"
            
#contrainte 6 Chaque jour, un poste roulant est occupé par au plus une personne
for p in range(0,cardP):
    for j in range(0,cardJ):
        if o[j] == 1 and (rho[p] == 0 or fac[p] == 1):
            prob += sum(x[i,j,p] for i in range(0,cardO)) <= 1, f"C52_{j,p}"

#contrainte 7 Chaque semaine, chaque opérateur est affecté à un poste

for i in range(0,cardO):
    for s in range(0,cardS):
        prob += sum(z[i,s,p] for p in range(0,cardP)) <= 1, f"C6_{i,s}"

#contrainte 8 Chaque semaine, chaque poste est assuré par un opérateur

for p in range(0,cardP):
    for s in range(0,cardS):
        if fac[p] == 0:
            prob += sum(z[i,s,p] for i in range(0,cardO)) == 1, f"C7_{s,p}"
        else:
            prob += sum(z[i,s,p] for i in range(0,cardO)) <= 1, f"C7_{s,p}"

#contrainte 9 Chaque opérateur est affecté à une position dans le roulement

for i in range(0,cardO):
    prob += sum(y[i,r] for r in range(0,cardR)) == 1, f"C8_{i}"

#contrainte 10 Chaque position dans le roulement est associée à un opérateur

for r in range(0,cardR):
    prob += sum(y[i,r] for i in range(0,cardO)) == 1, f"C9_{r}"

#contrainte 11 Un opérateur peut changer de poste si l'horaire de ce dernier est compatible

for i in range(0,cardO):
    for s in range(0,cardS):
        for p in range(0,cardP):
            a = 0
            #prob += z[i,s,p] <= sum(sigma1[p][p2]*y[i,rFunction(s,p2)] for p2 in range(0,cardP)), f"C10_{i,s,p}"

#Si on change quelqu'un d'horaire, il est très insatisfait, même si pas compétent à la base
#Si quelqu'un change de poste, en gardant le même horaire, il est faiblement insatisfait si il avait la compétence
for i in range(0,cardO):
    for s in range(0,cardS):
        for p in range(0,cardP):
            for p2 in range(cardP):
                if p != p2:
                    if sigma[p][p2] == 0:
                        #Très insatisfait si horaires éloignés
                        #prob += u2[i,s] >= y[i,rFunction(s,p)] + z[i,s,p2] - 1, f"C11_{i,s,p,p2}"
                        a = 0
                    else:
                        #Si horaires proches, insat si il avait la compétence
                        if kappa[i][p] == 1:
                            #prob += u[i,s] >= y[i,rFunction(s,p)] + z[i,s,p2] - 1, f"C11_{i,s,p,p2}"
                            a = 0

                            
for i in range(0,cardO):
    for s in range(0,cardS):
        for p in range(0,cardP):
            for p2 in range(cardP):
                if p != p2:
                    if sigma[p][p2] == 0:
                        if rho[p] == 1 and rho[p2] == 0:
                            prob += u1[i,s] >= y[i,rFunction(s,p)] + z[i,s,p2] - 1, f"C11_1_{i,s,p,p2}"
                            prob += u2[i,s] >= y[i,rFunction(s,p)] + z[i,s,p2] - 1, f"C11_2_{i,s,p,p2}"
                        else:
                            prob += u2[i,s] >= y[i,rFunction(s,p)] + z[i,s,p2] - 1, f"C11_1_{i,s,p,p2}"
                                
                    else:
                        if rho[p] == 1 and rho[p2] == 0:
                            prob += u1[i,s] >= y[i,rFunction(s,p)] + z[i,s,p2] - 1, f"C11_1_{i,s,p,p2}"
                                
for i in range(cardO):
    for s in range(cardS):
        for p in range(cardP):
            r = rFunction(s,p)
            prob += u[i,s] >= kappa[i][p] + y[i,r] + (1 - z[i,s,p]) - 2, f"Ctest{i,s,p}"
                                

for p in range(cardP):
    for s in range(cardS):
        if fac[p] == 1:
            prob += posteVacant[p,s] >= 1 - sum(z[i,s,p] for i in range(cardO)), f"C12_{p,s}"

print("fin")

fin


## 2.4) Résolution du problème

In [34]:
import time

prob.writeLP("newModele36sem.lp")

#warmstart=prob.new_solution()
for i in range(cardO):
    for r in range(cardR):
        a = 0
        y[i,r].setInitialValue(tempY[i][r])

start = time.time()
#prob.add_mip_start(warmstart)

#start = time.time() 
prob.solve(CPLEX_PY(warmStart=True))
end = time.time()

duree = end - start
print()
print("Temps de résolution : " + str(duree))
print("Valeur objectif = ", value(prob.objective))

Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_Read_DataCheck                          1
1 of 1 MIP starts provided solutions.
MIP start '1' defined initial solution with objective 54.0000.
Tried aggregator 1 time.
MIP Presolve eliminated 25451 rows and 5403 columns.
MIP Presolve modified 1245 coefficients.
Reduced MIP has 23368 rows, 6405 columns, and 107254 nonzeros.
Reduced MIP has 6405 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.06 sec. (67.73 ticks)
Probing time = 0.01 sec. (6.50 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 23368 rows, 6405 columns, and 107254 nonzeros.
Reduced MIP has 6405 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.13 sec. (149.63 ticks)
Probing changed sense of 36 constraints.
Probing time = 0.04 sec. (17.75 ticks)
Clique table members: 54292.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threa

# 3) Affichage de la solution

In [18]:
print("posteVac")
for p in range(cardP):
    for s in range(cardS):
        if(posteVacant[p,s].varValue != 0):
            print("posteVacant[",p,",",s,"] =", posteVacant[p,s].varValue)

posteVac


In [32]:
a = 23
poste1 = 5
poste2 = 7
print(kappa[a][poste1], kappa[a][poste2], ":", rho[poste1], rho[poste2], ":", sigma[poste1][poste2])
print(kappa[0][14])

1 1 : 1 1 : 1
1


In [29]:
igrec = [14,1,26,0,31,27,6,12,18,20,15,11,25,28,29,17,2,19,9,3,30,32,22,4,7,35,8,23,34,21,24,33,10,16,5,13]
for i in range(cardO):
    if kappa[i][igrec[i]] == 0:
        print(i)

4
11
14
16
17
28
31
33
34
35


In [14]:
print("uis")
for s in range(0,cardS):
    for i in range(0,cardO):
        if(u[i,s].varValue != 0):
            print("u[",i,",",s,"] =", u[i,s].varValue)
        if(u2[i,s].varValue != 0):
            print("u2[",i,",",s,"] =", u[i,s].varValue)
            


print('Affectation aux roulements')
for i in range(cardO):
    for r in range(cardR):
        if(y[i,r].varValue != 0):
            print('y[',i,',',r,'] =', y[i,r].varValue)

print()
print('Affectations hebdomadaires')
for s in range(0,cardS):
    print('semaine',s)
    for i in range(0,cardO):
        for p in range(0,cardP):
            if z[i,s,p].varValue==1:
                print('semaine',s,',',i,'affecté au poste',p)
                
print('======================================================')

for p in range(0,cardP):
    print()
    if rho[p] == 1:
        print('poste',p,'non rouleur')
    else:
        print('poste',p,'rouleur')
    for j in range(0,cardJ):
        for i in range(0,cardO):
            if (x[i,j,p].varValue == 1):
                print(i,'occupe le poste',p,'le jour',j)
                
print('======================================================')

                
for i in range(0,cardO):
    print()
    print('planning de l\'opérateur',i)
    for j in range(0,cardJ):
        for p in range(0,cardP):
            if (x[i,j,p].varValue == 1):
                print('poste',p,'le jour',j)

uis
Affectation aux roulements
y[ 0 , 2 ] = 1.0
y[ 1 , 12 ] = 1.0
y[ 2 , 21 ] = 1.0
y[ 3 , 3 ] = 1.0
y[ 4 , 10 ] = 1.0
y[ 5 , 28 ] = 1.0
y[ 6 , 31 ] = 1.0
y[ 7 , 16 ] = 1.0
y[ 8 , 18 ] = 1.0
y[ 9 , 0 ] = 1.0
y[ 10 , 17 ] = 1.0
y[ 11 , 1 ] = 1.0
y[ 12 , 26 ] = 1.0
y[ 13 , 23 ] = 1.0
y[ 14 , 15 ] = 1.0
y[ 15 , 7 ] = 1.0
y[ 16 , 6 ] = 1.0
y[ 17 , 20 ] = 1.0
y[ 18 , 34 ] = 1.0
y[ 19 , 22 ] = 1.0
y[ 20 , 25 ] = 1.0
y[ 21 , 29 ] = 1.0
y[ 22 , 4 ] = 1.0
y[ 23 , 8 ] = 1.0
y[ 24 , 24 ] = 1.0
y[ 25 , 35 ] = 1.0
y[ 26 , 32 ] = 1.0
y[ 27 , 19 ] = 1.0
y[ 28 , 5 ] = 1.0
y[ 29 , 30 ] = 1.0
y[ 30 , 13 ] = 1.0
y[ 31 , 27 ] = 1.0
y[ 32 , 11 ] = 1.0
y[ 33 , 33 ] = 1.0
y[ 34 , 9 ] = 1.0
y[ 35 , 14 ] = 1.0

Affectations hebdomadaires
semaine 0
semaine 0 , 0 affecté au poste 33
semaine 0 , 1 affecté au poste 12
semaine 0 , 2 affecté au poste 16
semaine 0 , 3 affecté au poste 3
semaine 0 , 4 affecté au poste 10
semaine 0 , 5 affecté au poste 28
semaine 0 , 6 affecté au poste 13
semaine 0 , 7 affecté au poste

# 4) Extraction des données au format texte

In [22]:
with open("solutionTexteNewModele36sem", "w") as file_object:
        file_object.write(str(cardO) + "\n")           #cardO
        file_object.write(str(cardP) + "\n")           #cardP
        file_object.write(str(cardR) + "\n")           #cardR
        file_object.write(str(cardS) + "\n")           #cardS
        ligne = ""
        for p in range(0,cardP):
            ligne = ligne + str(rho[p])
        file_object.write(ligne + "\n")                #rho
        
        ligne = ""
        for p in range(cardP):
            ligne += str(fac[p])
        file_object.write(ligne + "\n")                #fac
        
        ligne = ""
        for j in range(0,cardJ):
            ligne = ligne + str(int(o[j]))
        file_object.write(ligne + "\n")                #o_j
        
        for i in range(0,cardO):
            ligne = ""
            for p in range(0,cardP):
                ligne = ligne + str(kappa[i][p])
            file_object.write(ligne + "\n")            #kappa_ip
            
        for p in range(0,cardP):
            ligne = ""
            for p2 in range(0,cardP):
                ligne = ligne + str(sigma[p][p2])
            file_object.write(ligne + "\n")            #sigma_pp'
            
        for i in range(0,cardO):
            ligne = ""
            for j in range(0,cardJ):
                ligne = ligne + str(delta[i][j])
            file_object.write(ligne + "\n")            #delta_ij
            
        file_object.write(str(poids1) + "\n")          #poids1
        file_object.write(str(poids2) + "\n")          #poids2
        file_object.write(str(poids3) + "\n")          #poids3
        file_object.write(str(poids4) + "\n")          #poids4
        
        for i in range(0,cardO):
            for j in range(0,cardJ):
                ligne = ""
                for p in range(0,cardP):
                    ligne = ligne + str(int(x[i,j,p].varValue))
                file_object.write(ligne + "\n")        #x_ijp
        
        for i in range(0,cardO):
            ligne = ""
            for r in range(0,cardR):
                ligne = ligne + str(int(y[i,r].varValue))
            file_object.write(ligne + "\n")            #y_ir

        for i in range(0,cardO):
            for s in range(0,cardS):
                ligne = ""
                for p in range(0,cardP):
                    ligne = ligne + str(int(z[i,s,p].varValue))
                file_object.write(ligne + "\n")        #z_isp
        
        file_object.write(str(int(value(prob.objective))) + "\n")

# 5) Extraction des données au format CSV

In [ ]:
import csv
with open('CSV/planningExact.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    jours = [""]
    for s in range(0,cardS):
        jours.extend(["lundi","mardi","mercredi","jeudi","vendredi","samedi","dimanche"])
    writer.writerow(jours)
    
    dates = [""]
    for j in range(0,cardJ):
        dates.append("jour "+str(j))
    writer.writerow(dates)
    
    for i in range(0,cardO):
        planningPerso = [nomsOperateurs.iloc[i,1] + " " + nomsOperateurs.iloc[i,2]]
        for j in range(0,cardJ):
            found = False
            for p in range(0,cardP):
                if x[i,j,p].varValue == 1:
                    found = True
                    planningPerso.append(nomsPostes.iloc[p])
            if not found:
                if o[j] == 0:
                    planningPerso.append("")
                else:
                    planningPerso.append("RA")
        writer.writerow(planningPerso)

In [ ]:
print("Fin de programme")